In [47]:
model_checkpoint = 't5-small'
model_path='../../data/models/sparql-translator-20-06-2021'
model_name='sparql-translator-20-06-2021'
ds_path='../../data/dataset/lc-quad-wikidata-20-06-2021'

In [2]:
from tqdm import tqdm

In [3]:
from datasets import load_dataset, load_metric, Dataset, load_from_disk
raw_datasets = load_from_disk(ds_path)

In [4]:
# Preprocessing

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

comet_ml is installed but `COMET_API_KEY` is not set.


In [6]:
print(raw_datasets['test']['translation'][0]['en'],tokenizer(raw_datasets['test']['translation'][0]['en']))

Who is the  {country} for {head of state} of {Mahmoud Abbas} {'input_ids': [2645, 19, 8, 3, 2, 17529, 2, 21, 3, 2, 3313, 13, 538, 2, 13, 3, 2, 329, 9, 107, 51, 1063, 26, 891, 4883, 2, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [7]:
prefix = "translate English to Sparql: "

In [8]:
max_input_length = 0 
max_target_length = 0
for d in tqdm(raw_datasets['train']['translation']):
    len_en = len(d['en'])
    len_qry = len(d['sparql'])
    if len_en > max_input_length: max_input_length=len_en
    if len_qry > max_target_length: max_target_length=len_qry

100%|██████████| 38513/38513 [00:00<00:00, 1705865.52it/s]


In [9]:
print(max_input_length, max_target_length)

248 224


In [10]:
source_lang = "en"
target_lang = "sparql"

def preprocess_function(examples):
    inputs = []
    targets= []
    for ex in examples["translation"]:
      inputs.append(prefix + ex[source_lang])
      targets.append(ex[target_lang])
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [12]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 38513
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 9547
    })
})

In [13]:
# Fine-tuning the model

In [14]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [15]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
)

In [16]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [17]:
import numpy as np
metric = load_metric("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [18]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [19]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shahriar (use `wandb login --relogin` to force relogin)


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=4815, training_loss=1.0389406737013644, metrics={'train_runtime': 2202.9278, 'train_samples_per_second': 2.186, 'total_flos': 130441632178176.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 2433191936, 'init_mem_gpu_alloc_delta': 242026496, 'init_mem_cpu_peaked_delta': 177958912, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 54706176, 'train_mem_gpu_alloc_delta': 733817344, 'train_mem_cpu_peaked_delta': 127909888, 'train_mem_gpu_peaked_delta': 1319769088})

In [20]:
trainer.save_model(model_path)

In [30]:
from transformers import pipeline

translator = pipeline(
    "translation_xx_to_yy",
    model=model2,
    tokenizer=tokenizer
)

In [46]:
translator('Who is marlin?')#.to("cpu")

[{'translation_text': 'Qui est marlin?'}]